In [1]:
import xml.etree.ElementTree as ET
import os as os
from PIL import Image
import numpy as np
import torch
import os
import cv2
import torch
import random
import numbers
import numpy as np
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torch import randperm
from torch._utils import _accumulate
import torchvision
from torchvision import datasets, transforms, models
import torchvision.transforms.functional as F
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import torch.nn as nn
import time
from skimage import io, transform
import sklearn.metrics as skm
from scipy.spatial import distance

In [2]:
def print_bounding_box(bbox):
    print("*-*-*-*-*-*-*-* Bounding Box *-*-*-*-*-*-*-*")
    print("Xmin : ",bbox[0][0]," Ymin : ",bbox[0][1])
    print("Xmax : ",bbox[1,0]," Ymax : ",bbox[1,1])
    print("Xcent : ",bbox[2,0]," Ycent : ",bbox[2,1])
    print("Xlen : ",bbox[3,0]," Ylen : ",bbox[3,1])

In [3]:
class ToTensor(object):
    
    def __call__(self, data_dict): 
        transformed_dict = dict()
        
        for key in data_dict.keys():
            
            if key == 'image':
                image = data_dict[key]
                image = F.to_tensor(image)
                transformed_dict[key] = image
            else:
                dtls = torch.FloatTensor(data_dict[key])
                transformed_dict[key] = dtls
                
        return transformed_dict

In [4]:
class Rescale(object):
    
    def __init__(self, output_size):
        
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, data_dict):
        
        transformed_dict = dict()
        
        #print("Rescale Bounding Box")
        
        image = data_dict['image']
        w, h = image.size
        
        #image.show()
        #print("Old iMage dimension : ",w,"x",h)
        
        if isinstance(self.output_size, int):        
            
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
                
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)
        
        for key in data_dict.keys():
            
            if key == 'image':
                img = F.resize(image, (new_h, new_w))
                
                #img.show()
                #print("new iMage dimension : ",new_w,"x",new_h)
                
                transformed_dict[key] = img
            else:
                bounding_box = data_dict[key]
                
                #print(bounding_box)
                
                for i, dtls in enumerate(bounding_box):
    
                    #print("=====================Old Bounding Box================")
                    #print_bounding_box(dtls)
            
                    bounding_box[i][0] = np.round(dtls[0] * np.array([new_w / w, new_h / h]), 0)
                    bounding_box[i][1] = np.round(dtls[1] * np.array([new_w / w, new_h / h]), 0)
                    bounding_box[i][3] = np.abs([dtls[0, 0]-dtls[1, 0], dtls[0, 1]-dtls[1, 1]])
                    bounding_box[i][2] = np.array([dtls[0, 0]+dtls[3, 0]/2, dtls[0, 1]+dtls[3, 1]/2])
                    
                    #print("=====================New Bounding Box================")
                    #print_bounding_box(bounding_box[i])

                transformed_dict[key] = bounding_box
        return transformed_dict

In [5]:
class RandomVerticalFlip(object):
    
    def __init__(self, p=0.5):
        
        self.p = p

    def __call__(self, data_dict):
        
        transformed_dict = dict()
        
        #print("Verticle Flip Bounding Box")
        
        image = data_dict['image']
        w, h = image.size
        
        #print("Flipped image dimension : ",w,"x",h)
        
        if random.random() < self.p:
            
            for key in data_dict.keys():
                
                if key == 'image':
                    image = F.vflip(image)
                    
                    #image.show()
                    
                    transformed_dict[key] = image
                else:
                    bounding_box = data_dict[key]
                    
                    #print(bounding_box)
                    
                    for i, dtls in enumerate(bounding_box):
                        
                        #print("=====================Old Bounding Box================")
                        #print_bounding_box(dtls)
                        
                        if dtls[2][0] > 0 and dtls[2][1] > 0:
                            bounding_box[i][0] = np.array([dtls[0, 0], h - dtls[1, 1]])
                            bounding_box[i][1] = np.array([dtls[1, 0], bounding_box[i][0, 1] + dtls[3, 1] ])
                            bounding_box[i][3] = np.abs([dtls[0, 0]-dtls[1, 0], dtls[0, 1]-dtls[1, 1]])
                            bounding_box[i][2] = np.array([dtls[0, 0]+dtls[3, 0]/2, dtls[0, 1]+dtls[3, 1]/2])
                            
                            #print("=====================New Bounding Box================")
                            #print_bounding_box(bounding_box[i])
                            
                    transformed_dict[key] = bounding_box
        else:
            return data_dict
        
        return transformed_dict

In [6]:
class RandomHorizontalFlip(object):
    
    def __init__(self, p=0.5):
        
        self.p = p

    def __call__(self, data_dict):
        
        transformed_dict = dict()
        
        #print("Horizontal Flip Bounding Box")
        
        image = data_dict['image']
        w, h = image.size
        
        #print("Flipped image dimension : ",w,"x",h)
        
        if random.random() < self.p:
            
            for key in data_dict.keys():
                
                if key == 'image':
                    image = F.hflip(image)
                    
                    #image.show()
                    
                    transformed_dict[key] = image
                else:
                    bounding_box = data_dict[key]
                    
                    #print(bounding_box)
                    
                    for i, dtls in enumerate(bounding_box):
                        
                        #print("=====================Old Bounding Box================")
                        #print_bounding_box(dtls)
                        
                        if dtls[2][0] > 0 and dtls[2][1] > 0:
                            bounding_box[i][0] = np.array([w - (dtls[0, 0] + dtls[3, 0]), dtls[0, 1]])
                            bounding_box[i][1] = np.array([bounding_box[i][0, 0] + dtls[3, 0], dtls[1, 1] ])
                            bounding_box[i][3] = np.abs([dtls[0, 0]-dtls[1, 0], dtls[0, 1]-dtls[1, 1]])
                            bounding_box[i][2] = np.array([dtls[0, 0]+dtls[3, 0]/2, dtls[0, 1]+dtls[3, 1]/2])
                            
                            #print("=====================New Bounding Box================")
                            #print_bounding_box(bounding_box[i])
                            
                    transformed_dict[key] = bounding_box
        else:
            return data_dict
        
        return transformed_dict

In [7]:
class Normalize(object):
    
    def __init__(self, mean, std):
    
        self.mean = mean
        self.std = std
        
    def __call__(self, data_dict):
        
        bounding_box = dict()
        
        for key in data_dict.keys():
            
            if key == 'image':
                image = data_dict[key]
                bounding_box[key] = F.normalize(image, self.mean, self.std)
            else:
                bounding_box[key] = data_dict[key]
                
        return bounding_box

In [8]:
class Lambda(object):

    def __init__(self, lambd):
        
        assert callable(lambd), repr(type(lambd).__name__) + \
            " object is not callable"
        self.lambd = lambd

    def __call__(self, img):
        
        return self.lambd(img)

    def __repr__(self):
        
        return self.__class__.__name__ + '()'

In [9]:
class ColorJitter(object):

    def __init__(self, brightness=0, contrast=0, saturation=0, hue=0):
        self.brightness = self._check_input(brightness, 'brightness')
        self.contrast = self._check_input(contrast, 'contrast')
        self.saturation = self._check_input(saturation, 'saturation')
        self.hue = self._check_input(hue, 'hue', center=0, bound=(-0.5, 0.5),
                                     clip_first_on_zero=False)

    def _check_input(self, value, name, center=1, bound=(0, float('inf')), clip_first_on_zero=True):
        if isinstance(value, numbers.Number):
            if value < 0:
                raise ValueError(
                    "If {} is a single number, it must be non negative.".format(name))
            value = [center - value, center + value]
            if clip_first_on_zero:
                value[0] = max(value[0], 0)
        elif isinstance(value, (tuple, list)) and len(value) == 2:
            if not bound[0] <= value[0] <= value[1] <= bound[1]:
                raise ValueError(
                    "{} values should be between {}".format(name, bound))
        else:
            raise TypeError(
                "{} should be a single number or a list/tuple with lenght 2.".format(name))

        if value[0] == value[1] == center:
            value = None
        return value

    @staticmethod
    def get_params(brightness, contrast, saturation, hue):

        tforms = []

        if brightness is not None:
            brightness_factor = random.uniform(brightness[0], brightness[1])
            tforms.append(
                Lambda(lambda img: F.adjust_brightness(img, brightness_factor)))

        if contrast is not None:
            contrast_factor = random.uniform(contrast[0], contrast[1])
            tforms.append(
                Lambda(lambda img: F.adjust_contrast(img, contrast_factor)))

        if saturation is not None:
            saturation_factor = random.uniform(saturation[0], saturation[1])
            tforms.append(
                Lambda(lambda img: F.adjust_saturation(img, saturation_factor)))

        if hue is not None:
            hue_factor = random.uniform(hue[0], hue[1])
            tforms.append(Lambda(lambda img: F.adjust_hue(img, hue_factor)))

        random.shuffle(tforms)
        transform = transforms.Compose(tforms)

        return transform

    def __call__(self, sample):

        items = dict()
        for key in sample.keys():
            if key == 'image':
                image = sample[key]
                transform = self.get_params(self.brightness, self.contrast,
                                    self.saturation, self.hue)
                items[key] =  transform(image)
            else:
                items[key] = sample[key]
        return items

In [10]:
class SegTransform(object):
    def __init__(self, size=(480,640)):

        self.size = height
        
    def __call__(self, img, lbl):
        
        img = F.resize(img, size)
        lbl = F.resize(lbl, size)

        flip = random.random()
        if (flip < 0.5):
            img = f.vflip(img)
            lbl = f.vflip(lbl)
            img = f.hflip(img)
            lbl = f.hflip(lbl)
 
#         #Random translation 0–2 pixels (fill rest with padding)
#         transX = random.randint(0, 2) 
#         transY = random.randint(0, 2)
            
#         input = ImageOps.expand(input, border=(transX,transY,0,0), fill=0)
#         target = ImageOps.expand(target, border=(transX,transY,0,0), fill=7) 
       
#         input = input.crop((0, 0, input.size[0]-transX, input.size[1]-transY))
#         target = target.crop((0, 0, target.size[0]-transX, target.size[1]-transY))
#         input = ToTensor()(input)
#         target = ToLabel()(target)
#         target = Relabel(255,7)(target)
        
        return F.to_tensor(img), F.to_tensor(lbl)